In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import h5py
from sklearn.model_selection import train_test_split as tts
import random

In [10]:
dbfile = open('../RML2016_10a/RML2016.10a_dict.pkl', 'rb')      
db = pickle.load(dbfile,encoding='latin1') 

BPSK = {}
QPSK = {}
PAM4 = {}
AM_DSB = {}
GFSK = {}
QAM64 = {}
AM_SSB = {}
_8PSK = {}
QAM16 = {}
WBFM = {}
CPFSK = {}

SNR = []

for keys in db: 
    
#     print(keys[0],keys[1])
    SNR.append(keys[1])
    
    if keys[0] == 'QPSK':
        QPSK[str(keys[1])] = db[keys]
    if keys[0] == 'BPSK':
        BPSK[str(keys[1])] = db[keys]
    if keys[0] == 'PAM4':
        PAM4[str(keys[1])] = db[keys] 
    if keys[0] == 'AM-DSB':
        AM_DSB[str(keys[1])] = db[keys]
    if keys[0] == 'GFSK':
        GFSK[str(keys[1])] = db[keys]
    if keys[0] == 'QAM64':
        QAM64[str(keys[1])] = db[keys]
    if keys[0] == 'AM-SSB':
        AM_SSB[str(keys[1])] = db[keys]
    if keys[0] == '8PSK':
        _8PSK[str(keys[1])] = db[keys]
    if keys[0] == 'QAM16':
        QAM16[str(keys[1])] = db[keys]
    if keys[0] == 'WBFM':
        WBFM[str(keys[1])] = db[keys]
    if keys[0] == 'CPFSK':
        CPFSK[str(keys[1])] = db[keys]
        
print(db[keys].shape)
dbfile.close() 


(1000, 2, 128)


In [8]:
SNR = list(set(SNR))
mod_schemes = [BPSK,QPSK,PAM4,AM_DSB,GFSK,QAM64,AM_SSB,_8PSK,QAM16,WBFM,CPFSK]
labels =['BPSK','QPSK','PAM4','AM_DSB','GFSK','QAM64','AM_SSB','_8PSK','QAM16','WBFM','CPFSK']
print(SNR)
feature_file = '../project_features/base_scrnn_features.h5'

[0, 2, 4, 6, 8, 10, 12, -20, -18, 14, 16, -16, 18, -14, -12, -10, -8, -6, -4, -2]


In [ ]:
def create_train_test_splits(train_size=0.9):
#     tr_data, ts_data,tr_labels,ts_labels,snr_vec_tr,snr_vec_ts = [],[],[],[],[],[]
    label = 0
    for mod in mod_schemes:
        for snr in SNR:
            print(labels[label],snr)
            tr_size = int(len(mod[str(snr)])*train_size)
            ts_size = len(mod[str(snr)]) - tr_size
            
            mod_data = np.array(mod[str(snr)])
            mod_data = mod_data.transpose(0,2,1)
            mod_tr, mod_ts = tts(mod_data,train_size=train_size)
            
            y_tr = np.zeros([tr_size,11])
            y_tr[:,label]=1
            y_ts = np.zeros([ts_size,11])
            y_ts[:,label]=1
            
            if snr == SNR[0] and label==0:
                tr_data = mod_tr
                ts_data = mod_ts
                tr_labels = y_tr
                ts_labels = y_ts
                snr_vec_tr = np.ones([tr_size,1])*snr
                snr_vec_ts = np.ones([ts_size,1])*snr
                
            else:
                tr_data = np.concatenate([tr_data,mod_tr],axis=0)
                ts_data = np.concatenate([ts_data,mod_ts],axis=0)
                tr_labels = np.concatenate([tr_labels,y_tr],axis=0)
                ts_labels = np.concatenate([ts_labels,y_ts],axis=0)
                snr_vec_tr = np.concatenate([snr_vec_tr,np.ones([tr_size,1])*snr],axis=0)
                snr_vec_ts = np.concatenate([snr_vec_ts,np.ones([ts_size,1])*snr],axis=0)

        label+=1
    
#     tr_data,ts_data = np.array(tr_data),np.array(ts_data)
#     tr_labels,ts_labels = np.array(tr_labels),np.array(ts_labels)
#     snr_vec_tr,snr_vec_ts = np.array(snr_vec_tr),np.array(snr_vec_ts)
    return tr_data, ts_data,tr_labels,ts_labels,snr_vec_tr,snr_vec_ts


def base_scrnn_features(featureFileName,x_tr,x_ts,y_tr,y_ts,snr_tr,snr_ts):

    hf=h5py.File(feature_file, 'w')
    
    tr_dset_x = hf.create_dataset("x_tr", (x_tr.shape[0],x_tr.shape[1],x_tr.shape[2]))
    tr_dset_y = hf.create_dataset("y_tr", (x_tr.shape[0],11))
    tr_dset_snr = hf.create_dataset("snr_tr", (x_tr.shape[0],1))

    ts_dset_x = hf.create_dataset("x_ts", (x_ts.shape[0],x_ts.shape[1],x_ts.shape[2]))
    ts_dset_y = hf.create_dataset("y_ts", (x_ts.shape[0],11))
    ts_dset_snr = hf.create_dataset("snr_ts", (x_ts.shape[0],1))
    
    tr_dset_x[:] = x_tr
    ts_dset_x[:] = x_ts
    tr_dset_y[:] = y_tr
    ts_dset_y[:] = y_ts
    tr_dset_snr[:] = snr_tr
    ts_dset_snr[:] = snr_ts
    
    hf.close()
    
def amp_phs_features(featureFileName,x_tr,x_ts,y_tr,y_ts,snr_tr,snr_ts):

    hf=h5py.File(feature_file, 'w')
    
    tr_dset_x = hf.create_dataset("x_tr", (x_tr.shape[0],x_tr.shape[1],x_tr.shape[2]))
    tr_dset_y = hf.create_dataset("y_tr", (x_tr.shape[0],11))
    tr_dset_snr = hf.create_dataset("snr_tr", (x_tr.shape[0],1))

    ts_dset_x = hf.create_dataset("x_ts", (x_ts.shape[0],x_ts.shape[1],x_ts.shape[2]))
    ts_dset_y = hf.create_dataset("y_ts", (x_ts.shape[0],11))
    ts_dset_snr = hf.create_dataset("snr_ts", (x_ts.shape[0],1))
    
    amp = np.sqrt(np.sum(np.square(x_tr),axis=-1))
    phs = np.arctan(x_tr[:,:,1]/x[:,:,0])
    
    X_tr = 
    
    tr_dset_x[:] = x_tr
    ts_dset_x[:] = x_ts
    tr_dset_y[:] = y_tr
    ts_dset_y[:] = y_ts
    tr_dset_snr[:] = snr_tr
    ts_dset_snr[:] = snr_ts
    
    hf.close()

In [5]:
tr_data, ts_data,tr_labels,ts_labels,snr_vec_tr,snr_vec_ts = create_train_test_splits(0.9)
tr_data[tr_data==0]=1e-8
ts_data[ts_data==0]=1e-8

BPSK 0
BPSK 2
BPSK 4
BPSK 6
BPSK 8
BPSK 10
BPSK 12
BPSK -20
BPSK -18
BPSK 14
BPSK 16
BPSK -16
BPSK 18
BPSK -14
BPSK -12
BPSK -10
BPSK -8
BPSK -6
BPSK -4
BPSK -2
QPSK 0
QPSK 2
QPSK 4
QPSK 6


/home/jeel/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


QPSK 8
QPSK 10
QPSK 12
QPSK -20
QPSK -18
QPSK 14
QPSK 16
QPSK -16
QPSK 18
QPSK -14
QPSK -12
QPSK -10
QPSK -8
QPSK -6
QPSK -4
QPSK -2
PAM4 0
PAM4 2
PAM4 4
PAM4 6
PAM4 8
PAM4 10
PAM4 12
PAM4 -20
PAM4 -18
PAM4 14
PAM4 16
PAM4 -16
PAM4 18
PAM4 -14
PAM4 -12
PAM4 -10
PAM4 -8
PAM4 -6
PAM4 -4
PAM4 -2
AM_DSB 0
AM_DSB 2
AM_DSB 4
AM_DSB 6
AM_DSB 8
AM_DSB 10
AM_DSB 12
AM_DSB -20
AM_DSB -18
AM_DSB 14
AM_DSB 16
AM_DSB -16
AM_DSB 18
AM_DSB -14
AM_DSB -12
AM_DSB -10
AM_DSB -8
AM_DSB -6
AM_DSB -4
AM_DSB -2
GFSK 0
GFSK 2
GFSK 4
GFSK 6
GFSK 8
GFSK 10
GFSK 12
GFSK -20
GFSK -18
GFSK 14
GFSK 16
GFSK -16
GFSK 18
GFSK -14
GFSK -12
GFSK -10
GFSK -8
GFSK -6
GFSK -4
GFSK -2
QAM64 0
QAM64 2
QAM64 4
QAM64 6
QAM64 8
QAM64 10
QAM64 12
QAM64 -20
QAM64 -18
QAM64 14
QAM64 16
QAM64 -16
QAM64 18
QAM64 -14
QAM64 -12
QAM64 -10
QAM64 -8
QAM64 -6
QAM64 -4
QAM64 -2
AM_SSB 0
AM_SSB 2
AM_SSB 4
AM_SSB 6
AM_SSB 8
AM_SSB 10
AM_SSB 12
AM_SSB -20
AM_SSB -18
AM_SSB 14
AM_SSB 16
AM_SSB -16
AM_SSB 18
AM_SSB -14
AM_SSB -12
AM_SSB -10
AM

In [15]:
tr_data.shape

(198000, 128, 2)

In [6]:
z = np.arange(len(tr_data))
random.shuffle(z)
tr_data,tr_labels,snr_vec_tr = tr_data[z],tr_labels[z],snr_vec_tr[z]

In [7]:
base_scrnn_features(feature_file,tr_data,ts_data,tr_labels,ts_labels,snr_vec_tr,snr_vec_ts)